In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

* 지니 차트 TOP 200

In [3]:
url='https://www.genie.co.kr/chart/top200'
req=requests.get(url)
html = req.text
html

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [4]:
# https://blog.hangyeong.com/1009  user-agent값 확인하는방법
#Chrome User-Agetnt
header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
req=requests.get(url,headers=header)
soup=BeautifulSoup(req.text)

In [ ]:
soup

찾으려고 하는 데이터의 노드 찾기

In [ ]:
trs=soup.select('tr.list')
len(trs)

50

In [ ]:
#개발자 도구에서 찾아주는 selector 
trs=soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)

50

In [ ]:
tr=trs[12]
tr

In [ ]:
#rank
tr.select_one('.number').get_text()

'13\n                                        \n                                    \n유지\n\n'

In [ ]:
try:
  rank=int(tr.select_one('.number').get_text().split('\n')[0])
except:
  rank=0

rank

13

In [ ]:
#제목
try:   #일반적인 노래
  title = tr.select_one('.title.ellipsis').string.strip()
except:   #19금노래
  title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()

title

'INVU'

In [ ]:
#artist
artist=tr.select_one('.artist.ellipsis').text.strip()
artist

'태연 (TAEYEON)'

In [ ]:
#album
album=tr.select_one('.albumtitle.ellipsis').text.strip()
album

'INVU - The 3rd Album'

한 페이지 내에 있는 정보 가져오기

In [ ]:
rank_list,title_list,artist_list,album_list=[],[],[],[]

for i,tr in enumerate(trs):
  try:
    rank=int(tr.select_one('.number').get_text().split('\n')[0])
  except:
    rank=0
  try:   #일반적인 노래
    title = tr.select_one('.title.ellipsis').string.strip()
  except:   #19금노래
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
  artist=tr.select_one('.artist.ellipsis').text.strip()
  album=tr.select_one('.albumtitle.ellipsis').text.strip()
  rank_list.append(rank)
  title_list.append(title)
  artist_list.append(artist)
  album_list.append(album)

len(album_list)

50

In [ ]:
lines=[]
for i,tr in enumerate(trs):
  try:
    rank=int(tr.select_one('.number').get_text().split('\n')[0])
  except:
    rank=0
  try:   #일반적인 노래
    title = tr.select_one('.title.ellipsis').string.strip()
  except:   #19금노래
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
  artist=tr.select_one('.artist.ellipsis').text.strip()
  album=tr.select_one('.albumtitle.ellipsis').text.strip()
  line={'순위':rank,'제목':title,'아티스트':artist,'앨범':album}
  lines.append(line)
lines[0]

{'순위': 1,
 '아티스트': '싸이 (Psy)',
 '앨범': '싸다9',
 '제목': 'That That (Prod. & Feat. SUGA of BTS)'}

데이터 프레임 만들기

In [ ]:
df=pd.DataFrame(lines)
df.head()

,순위,제목,아티스트,앨범
0,1,That That (Prod. & Feat. SUGA of BTS),싸이 (Psy),싸다9
1,2,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
2,3,LOVE DIVE,IVE (아이브),LOVE DIVE
3,4,TOMBOY,(여자)아이들,I NEVER DIE
4,5,사랑인가 봐,멜로망스 (MeloMance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)


In [ ]:
#다른버전
df=pd.DataFrame({'순위':rank_list,'제목':title_list,'아티스트':artist_list,'앨범':album_list})
df.head()

,순위,제목,아티스트,앨범
0,1,That That (Prod. & Feat. SUGA of BTS),싸이 (Psy),싸다9
1,2,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
2,3,LOVE DIVE,IVE (아이브),LOVE DIVE
3,4,TOMBOY,(여자)아이들,I NEVER DIE
4,5,사랑인가 봐,멜로망스 (MeloMance),사랑인가 봐 (사내맞선 OST 스페셜 트랙)


모든 페이지에 대해 가져오기

In [ ]:
lines=[]
for page in range(1,5):
  url=f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220502&hh=15&rtm=Y&pg={page}'
  header={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
  req=requests.get(url,headers=header)
  soup=BeautifulSoup(req.text)
  trs=soup.select('tr.list')

  for i,tr in enumerate(trs):
    try:
      rank=int(tr.select_one('.number').get_text().split('\n')[0])
    except:
      rank=0
    try:   #일반적인 노래
      title = tr.select_one('.title.ellipsis').string.strip()
    except:   #19금노래
      title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    artist=tr.select_one('.artist.ellipsis').text.strip()
    album=tr.select_one('.albumtitle.ellipsis').text.strip()
    line={'순위':rank,'제목':title,'아티스트':artist,'앨범':album}
    lines.append(line)

df=pd.DataFrame(lines)
df.tail()

,순위,제목,아티스트,앨범
195,196,비 오는 날 듣기 좋은 노래 (Feat. Colde),에픽하이 (EPIK HIGH),비 오는 날 듣기 좋은 노래
196,197,우산이 없어요,김호중,우리家
197,198,O.O,NMIXX,AD MARE
198,199,봄 사랑 벚꽃 말고,HIGH4 & 아이유,봄 사랑 벚꽃 말고
199,200,안녕 (Hello),조이 (JOY),안녕 (Hello) - Special Album


In [ ]:
df.to_csv('지니차트Top200.csv',index=False)

In [ ]:
#한빛 출판사 (제목,저자,출간일,쪽수,가격,역자)